# ***`Homework`***
---

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
jan23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

## ***Q1. Downloading the data***
---

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- 19

In [4]:
print(f"Number of columns: {jan23.shape[1]}")

Number of columns: 19


## Q2. Computing duration
---

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 32.59
- 42.59
- 52.59
- 62.59

In [5]:
# Calculate duration in minutes
jan23['duration_minutes'] = (jan23['tpep_dropoff_datetime'] - jan23['tpep_pickup_datetime']).dt.total_seconds() / 60
feb23['duration_minutes'] = (feb23['tpep_dropoff_datetime'] - feb23['tpep_pickup_datetime']).dt.total_seconds() / 60

jan23['duration_minutes'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration_minutes, dtype: float64

## ***Q3. Dropping outliers***
---

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98%

In [6]:
# Filter out trips that are less than 1 minute and more than 60 minutes
mask_jan = (jan23['duration_minutes'] >= 1) & (jan23['duration_minutes'] <= 60)
jan23_clean = jan23[mask_jan].copy()

mask_feb = (feb23['duration_minutes'] >= 1) & (feb23['duration_minutes'] <= 60)
feb23_clean = feb23[mask_feb].copy()

# Calculate and print the fraction of trips that are between 1 and 60 minutes
fraction = jan23_clean.shape[0] / jan23.shape[0]
print(f"Fraction of trips that are between 1 and 60 minutes: {fraction:.2f}")

Fraction of trips that are between 1 and 60 minutes: 0.98


## ***Q4. One-hot encoding***
---

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- 515
- 715

In [7]:
# Filter columns
columns = ['PULocationID', 'DOLocationID', 'duration_minutes']
jan23_clean = jan23_clean[columns]
feb23_clean = feb23_clean[columns]

# Cast to str
jan23_clean['PULocationID'] = jan23_clean['PULocationID'].astype(str)
jan23_clean['DOLocationID'] = jan23_clean['DOLocationID'].astype(str)
feb23_clean['PULocationID'] = feb23_clean['PULocationID'].astype(str)
feb23_clean['DOLocationID'] = feb23_clean['DOLocationID'].astype(str)

# Define train and validation sets
X_train, y_train = jan23_clean.drop(columns='duration_minutes'), jan23_clean['duration_minutes']
X_test, y_test = feb23_clean.drop(columns='duration_minutes'), feb23_clean['duration_minutes']

# Vectorize
dv = DictVectorizer(sparse=False)

train_dict = X_train.to_dict(orient='records')
X_train_encoded = dv.fit_transform(train_dict)

test_dict = X_test.to_dict(orient='records')
X_test_encoded = dv.transform(test_dict)

# Print number of columns
print(f"Number of columns: {X_train_encoded.shape[1]}")

Number of columns: 515


In [8]:
X_train_encoded[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## ***Q5. Training a model***
---

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

- 3.64
- 7.64
- 11.64
- 16.64

In [9]:
# Train and evaluate the model
model = LinearRegression()
model.fit(X_train_encoded, y_train)

# Predict
y_pred_train = model.predict(X_train_encoded)

# Evaluate
rmse = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))
train_rmse = rmse(y_train, y_pred_train)
print(f"Train RMSE: {train_rmse:.2f}")

# Predict and evaluate on test set
y_pred_test = model.predict(X_test_encoded)
test_rmse = rmse(y_test, y_pred_test)
print(f"Test RMSE: {test_rmse:.2f}")

: 

## ***Q6. Evaluating the model***
---

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

- 3.81
- 7.81
- 11.81
- 16.81

In [ ]:
y_pred_test = model.predict(X_test_encoded)
rmse(y_test, y_pred_test)

: 

# Alternative to eventually solve kernel crash:

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the data
jan23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

# Calculate duration in minutes
jan23['duration_minutes'] = (jan23['tpep_dropoff_datetime'] - jan23['tpep_pickup_datetime']).dt.total_seconds() / 60
feb23['duration_minutes'] = (feb23['tpep_dropoff_datetime'] - feb23['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter out trips that are less than 1 minute and more than 60 minutes
mask_jan = (jan23['duration_minutes'] >= 1) & (jan23['duration_minutes'] <= 60)
jan23_clean = jan23[mask_jan].copy()

mask_feb = (feb23['duration_minutes'] >= 1) & (feb23['duration_minutes'] <= 60)
feb23_clean = feb23[mask_feb].copy()

# Calculate and print the fraction of trips that are between 1 and 60 minutes
fraction = jan23_clean.shape[0] / jan23.shape[0]
print(f"Fraction of trips that are between 1 and 60 minutes: {fraction:.2f}")

# Filter columns
columns = ['PULocationID', 'DOLocationID', 'duration_minutes']
jan23_clean = jan23_clean[columns]
feb23_clean = feb23_clean[columns]

# Define train and validation sets
X_train, y_train = jan23_clean.drop(columns='duration_minutes'), jan23_clean['duration_minutes']
X_test, y_test = feb23_clean.drop(columns='duration_minutes'), feb23_clean['duration_minutes']

# OneHotEncode categorical features
encoder = OneHotEncoder(handle_unknown='ignore')  # Handle unknown categories

X_train_encoded = encoder.fit_transform(X_train[['PULocationID', 'DOLocationID']])
X_test_encoded = encoder.transform(X_test[['PULocationID', 'DOLocationID']])

# Convert to dense array
X_train_encoded = X_train_encoded.toarray()
X_test_encoded = X_test_encoded.toarray()

# Train and evaluate the model
model = LinearRegression()
model.fit(X_train_encoded, y_train)

# Predict and evaluate on the training set
y_pred_train = model.predict(X_train_encoded)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"Train RMSE: {train_rmse:.2f}")

# Predict and evaluate on the test set
y_pred_test = model.predict(X_test_encoded)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"Test RMSE: {test_rmse:.2f}")


Fraction of trips that are between 1 and 60 minutes: 0.98


In [8]:
def load_and_convert_data(file_path, columns):
    # Load data
    data = pd.read_parquet(file_path, columns=columns, engine='pyarrow')
    
    # Convert float64 columns to float32
    for col in data.select_dtypes(include=['float64']).columns:
        data[col] = data[col].astype('float16')
    
    return data

In [9]:
import pandas as pd
import numpy as np
import logging
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the data
logging.info("Loading January data")

cols_to_read = ['tpep_dropoff_datetime', 'tpep_pickup_datetime', 'PULocationID', 'DOLocationID']

jan23 = load_and_convert_data(
    file_path='https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet',
    columns=cols_to_read
)
logging.info("Loading February data")
feb23 = load_and_convert_data(
    file_path='https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet',
    columns=cols_to_read
)

# Calculate duration in minutes
logging.info("Calculating duration in minutes for January data")
jan23['duration_minutes'] = (jan23['tpep_dropoff_datetime'] - jan23['tpep_pickup_datetime']).dt.total_seconds() / 60
logging.info("Calculating duration in minutes for February data")
feb23['duration_minutes'] = (feb23['tpep_dropoff_datetime'] - feb23['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter out trips that are less than 1 minute and more than 60 minutes
logging.info("Filtering January data")
mask_jan = (jan23['duration_minutes'] >= 1) & (jan23['duration_minutes'] <= 60)
jan23_clean = jan23[mask_jan].copy()

logging.info("Filtering February data")
mask_feb = (feb23['duration_minutes'] >= 1) & (feb23['duration_minutes'] <= 60)
feb23_clean = feb23[mask_feb].copy()

# Calculate and print the fraction of trips that are between 1 and 60 minutes
fraction = jan23_clean.shape[0] / jan23.shape[0]
logging.info(f"Fraction of trips that are between 1 and 60 minutes: {fraction:.2f}")

# Filter columns
logging.info("Filtering columns")
columns = ['PULocationID', 'DOLocationID', 'duration_minutes']
jan23_clean = jan23_clean[columns]
feb23_clean = feb23_clean[columns]

# Define train and validation sets
logging.info("Defining train and test sets")
X_train, y_train = jan23_clean.drop(columns='duration_minutes'), jan23_clean['duration_minutes']
X_test, y_test = feb23_clean.drop(columns='duration_minutes'), feb23_clean['duration_minutes']

# OneHotEncode categorical features
logging.info("One-hot encoding categorical features")
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)

X_train_encoded = encoder.fit_transform(X_train[['PULocationID', 'DOLocationID']])
X_test_encoded = encoder.transform(X_test[['PULocationID', 'DOLocationID']])

# Convert to dense array
#logging.info("Converting to dense arrays")
#X_train_encoded = X_train_encoded.toarray()
#X_test_encoded = X_test_encoded.toarray()

# Train the model
logging.info("Training the model")
try: 
    model = LinearRegression()
    model.fit(X_train_encoded, y_train)
except Exception as e:
    logging.error("Operation failed:", exc_info=True)

# Predict and evaluate on the training set
logging.info("Making predictions on the training set")
y_pred_train = model.predict(X_train_encoded)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
logging.info(f"Train RMSE: {train_rmse:.2f}")

# Predict and evaluate on the test set
logging.info("Making predictions on the test set")
y_pred_test = model.predict(X_test_encoded)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
logging.info(f"Test RMSE: {test_rmse:.2f}")

2024-05-15 09:59:10,757 - INFO - Loading January data
2024-05-15 09:59:11,269 - INFO - Loading February data
2024-05-15 09:59:11,557 - INFO - Calculating duration in minutes for January data
2024-05-15 09:59:11,659 - INFO - Calculating duration in minutes for February data
2024-05-15 09:59:11,721 - INFO - Filtering January data
2024-05-15 09:59:11,947 - INFO - Filtering February data
2024-05-15 09:59:12,134 - INFO - Fraction of trips that are between 1 and 60 minutes: 0.98
2024-05-15 09:59:12,135 - INFO - Filtering columns
2024-05-15 09:59:12,180 - INFO - Defining train and test sets
2024-05-15 09:59:12,207 - INFO - One-hot encoding categorical features
/home/basti/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2024-05-15 09:59:13,774 - INFO - Training the m